In [1]:
import numpy as np
import pandas as pd
import operator
import os
import string
import re
import random
import sys
import platform
import json
import shutil

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from PIL import Image

from transformers import XLMRobertaTokenizer

from tqdm.auto import tqdm

c:\Users\guts4\anaconda3\envs\deepdaiv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
CFG = {
    # 'EPOCHS': 30,
    # 'LEARNING_RATE': 2e-5,
    # 'BATCH_SIZE': 32,
    'SEED': 42,
    # 'MAX_LEN': 16,
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [2]:
from datasets import OKVQADataset
from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer("./dataset/beit3.spm")

OKVQADataset.make_dataset_index(
    data_path="./dataset",
    tokenizer=tokenizer,
    annotation_data_path="./dataset/vqa",
)

not all images have caption annotations
8998 8372 8372
Write ./dataset\vqa.train.jsonl with 8381 items !
not all images have caption annotations
40504 4784 4784
Write ./dataset\vqa.val.jsonl with 4797 items !
Contains 4784 image and 4797 pairs for val set!
Write ./dataset\vqa.trainable_val.jsonl with 3796 items !
Write ./dataset\vqa.rest_val.jsonl with 1001 items !
Write ./dataset\vqa.test.jsonl with 4797 items !


In [ ]:
python -m torch.distributed.launch --nproc_per_node=1 run_beit3_finetuning.py --model beit3_base_patch16_480 --input_size 480 --task vqav2 --batch_size 16 --layer_decay 1.0 --lr 3e-5 --update_freq 1 --randaug --epochs 10 --warmup_epochs 1 --drop_path 0.1 --sentencepiece_model C:/Users/guts4/unilm/beit3/dataset/beit3.spm --finetune C:/Users/guts4/unilm/beit3/dataset/beit3_base_patch16_224.pth --data_path C:/Users/guts4/unilm/beit3/dataset --output_dir C:/Users/guts4/unilm/beit3 --log_dir C:/Users/guts4/unilm/beit3/log --weight_decay 0.01 --seed 42 --save_ckpt_freq 5 --task_head_lr_weight 20 --opt_betas 0.9 0.98 --enable_deepspeed

In [ ]:
python -m torch.distributed.launch --nproc_per_node=1 run_beit3_finetuning.py --model beit3_base_patch16_480 --input_size 480 --task vqav2 --batch_size 16 --sentencepiece_model C:/Users/guts4/unilm/beit3/dataset/beit3.spm --finetune C:/Users/guts4/unilm/beit3/dataset/beit3_base_patch16_224.pth --data_path C:/Users/guts4/unilm/beit3/dataset --output_dir C:/Users/guts4/unilm/beit3 --eval --dist_eval


In [ ]:
torchrun --nproc_per_node=1 run_beit3_finetuning.py `
     --model beit3_base_patch16_480 `
     --input_size 480 `
     --task vqav2 `
     --batch_size 16 `
     --layer_decay 1.0 `
     --lr 3e-5 `
     --update_freq 1 `
     --randaug `
     --epochs 10 `
     --warmup_epochs 1 `
     --drop_path 0.1 `
     --sentencepiece_model C:/Users/guts4/unilm/beit3/dataset/beit3.spm `
     --finetune C:/Users/guts4/unilm/beit3/dataset/beit3_base_patch16_224.pth `
     --data_path C:/Users/guts4/unilm/beit3/dataset `
     --output_dir C:/Users/guts4/unilm/beit3 `
     --log_dir C:/Users/guts4/unilm/beit3/log `
     --weight_decay 0.01 `
     --seed 42 `
     --save_ckpt_freq 5 `
     --task_head_lr_weight 20 `
     --opt_betas 0.9 0.98 `
     --enable_deepspeed


In [ ]:
python -m torch.distributed.launch --nproc_per_node=1 run_beit3_finetuning.py --model beit3_base_patch16_480 --input_size 480 --sentencepiece_model C:/Users/guts4/unilm/beit3/dataset/beit3.spm --task vqav2 \
        --batch_size 8 \
        --layer_decay 1.0 \
        --lr 3e-5 \
        --update_freq 1 \
        --randaug \
        --epochs 10 \
        --warmup_epochs 1 \
        --drop_path 0.1 \
        --sentencepiece_model C:/Users/guts4/unilm/beit3/dataset/beit3.spm \
        --finetune C:/Users/guts4/unilm/beit3/dataset/beit3_base_patch16_224.pth \
        --data_path C:/Users/guts4/unilm/beit3/dataset \
        --output_dir C:/Users/guts4/unilm/beit3 \
        --log_dir C:/Users/guts4/unilm/beit3/log \
        --weight_decay 0.01 \
        --seed 42 \
        --save_ckpt_freq 5 \
        --task_head_lr_weight 20 \
        --opt_betas 0.9 0.98 \
        --enable_deepspeed